<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/%D1%81%D1%82%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B8%20/%D0%90%D0%B9%D0%A2%D0%B8%D0%9E%D0%BD_%D0%B7%D0%B0%D0%BC%D0%B5%D0%BD%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%D1%8B%D1%85_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%BE%D0%B2/9_Recognition_Speech__To_Replace__Text_Filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SpeechRecognition
!pip install pydub
!pip install jiwer
!pip install vosk
!pip install librosa soundfile noisereduce
!pip install noisereduce
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-7ddbp1fa
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-7ddbp1fa
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import zipfile
import os
import speech_recognition as sr
import time
import pandas as pd
from pydub import AudioSegment
import shutil
import math
import tempfile
import vosk
import numpy as np
import librosa
import soundfile as sf
import noisereduce as nr
import gc
import jiwer
import whisper

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


ЗАГРУЗКА ДАННЫХ
РАСПАКОВЫВАЕМ АРХИВ ЗВУКОВЫХ ДОРОЖЕК

In [ ]:
# Путь к архиву и директории, где будут распакованы файлы
zip_file_path = '/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Voice_filters/Dataset/Датасет_аудио (1).zip'
extracted_path = '/content/drive/MyDrive/Audio/extracted1/'

# Создаем директорию, если она не существует
os.makedirs(extracted_path, exist_ok=True)

# Распаковываем архив
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

print("Архив успешно распакован в", extracted_path)

Архив успешно распакован в /content/drive/MyDrive/Audio/extracted1/


ПЕРЕИМЕНОВЫВАЕМ ВСЕ ФАЙЛЫ (001, 002 и тд.)

In [ ]:
# Путь к директории с распакованными файлами
extracted_path = '/content/drive/MyDrive/Audio/extracted1/Датасет_аудио1'

# Получаем список файлов в директории
audio_files = [f for f in os.listdir(extracted_path) if f.endswith(".ogg")]

# Проходимся по каждому файлу и пронумеровываем
for i, audio_file in enumerate(audio_files, start=1):
    old_path = os.path.join(extracted_path, audio_file)
    new_name = f"{i:03d}.ogg"  # Форматирование имени файла, чтобы числа были трехзначные
    new_path = os.path.join(extracted_path, new_name)

    # Переименовываем файл
    os.rename(old_path, new_path)

print("Файлы успешно пронумерованы.")


Файлы успешно пронумерованы.


КОНВЕРТИРУЕМ В ФОРМАТ WAV ДЛЯ РАБОТЫ С БИБЛИОТЕКОЙ SPEECH_RECOGNITION

In [ ]:
numbered_files_path = '/content/drive/MyDrive/Audio/extracted1/Датасет_аудио1'
wav_files_path = '/content/drive/MyDrive/Audio/wav1/'

# Если папка существует, удаляем её
if os.path.exists(wav_files_path):
    shutil.rmtree(wav_files_path)

# Создаем директорию для WAV файлов
os.makedirs(wav_files_path, exist_ok=True)

# Получаем список файлов в директории
audio_files = [f for f in os.listdir(numbered_files_path) if f.endswith(".ogg")]

total_time = 0

# Проходимся по каждому файлу и конвертируем в WAV
for audio_file in audio_files:
    try:
        # Засекаем время начала преобразования файла
        start_time = time.time()

        audio_path = os.path.join(numbered_files_path, audio_file)

        # Загружаем аудиофайл OGG и конвертируем его в WAV
        audio = AudioSegment.from_ogg(audio_path)
        wav_filename = os.path.basename(audio_file).rsplit('.', 1)[0] + ".wav"
        wav_path = os.path.join(wav_files_path, wav_filename)
        audio.export(wav_path, format="wav")

        # Засекаем время завершения преобразования файла
        end_time = time.time()
        conversion_time = end_time - start_time
        total_time += conversion_time

        # Выводим время преобразования файла
        print(f"Файл {audio_file} преобразован за {conversion_time:.2f} секунд")

    except Exception as e:
        print(f"Ошибка при обработке файла {audio_file}: {e}")

# Считаем среднее время преобразования
if audio_files:
    average_time = total_time / len(audio_files)
    print(f"Среднее время преобразования одного файла: {average_time:.2f} секунд")
else:
    print("Нет файлов для преобразования.")

print("Конвертация завершена. WAV файлы сохранены в", wav_files_path)

Файл 001.ogg преобразован за 0.39 секунд
Файл 002.ogg преобразован за 0.29 секунд
Файл 003.ogg преобразован за 0.34 секунд
Файл 004.ogg преобразован за 0.28 секунд
Файл 005.ogg преобразован за 0.27 секунд
Файл 006.ogg преобразован за 1.80 секунд
Файл 007.ogg преобразован за 0.40 секунд
Файл 008.ogg преобразован за 0.34 секунд
Файл 009.ogg преобразован за 0.38 секунд
Файл 010.ogg преобразован за 0.39 секунд
Файл 011.ogg преобразован за 0.48 секунд
Файл 012.ogg преобразован за 0.28 секунд
Файл 013.ogg преобразован за 0.28 секунд
Файл 014.ogg преобразован за 0.29 секунд
Файл 015.ogg преобразован за 0.32 секунд
Файл 016.ogg преобразован за 0.34 секунд
Файл 017.ogg преобразован за 0.25 секунд
Файл 018.ogg преобразован за 0.31 секунд
Файл 019.ogg преобразован за 0.29 секунд
Файл 020.ogg преобразован за 0.24 секунд
Файл 021.ogg преобразован за 0.29 секунд
Файл 022.ogg преобразован за 0.27 секунд
Файл 023.ogg преобразован за 0.29 секунд
Файл 024.ogg преобразован за 0.38 секунд
Файл 025.ogg пре

РАСПОЗНОВАНИЕ АУДИО (GOOGLE SPEECH RECOGNITION)

ФУНКЦИЯ ОПРЕДЕЛЕНИЯ ДЛИНЫ ФАЙЛА

In [ ]:

def get_duration(file_path):
    audio = AudioSegment.from_file(file_path)
    duration_in_seconds = len(audio) / 1000  # Преобразование из миллисекунд в секунды
    return duration_in_seconds

РАСПОЗНАЕМ АУДИО ФАЙЛЫ И ЗАПИСЫВАЕМ В ИДЕНТИЧНЫЕ ПО № ТЕКСТОВЫЕ ФАЙЛЫ

In [ ]:

audio_folder = '/content/drive/MyDrive/Audio/wav1'
text_files_path = '/content/drive/MyDrive/Audio/extracted1/speech_recognition1/'
os.makedirs(text_files_path, exist_ok=True)

total_time = 0
file_number = 0

for i in range(1, 503):
    input_file = os.path.join(audio_folder, f"{i:03d}.wav")
    output_file = os.path.join(text_files_path, f"{i:03d}.txt")
    file_number += 1

    # Засекаем время начала обработки файла
    start_time = time.time()

    # Читаем файл
    with open(input_file, 'rb') as f:
        data = f.read()

    # Проверяем размер и длительность файла
    file_size = os.path.getsize(input_file)
    file_duration = get_duration(input_file)

    try:
        # Если файл слишком большой или длинный, делим его на отрезки по 30 секунд
        if file_size > 1024 * 1024 or file_duration > 30:
            segment_duration = 30  # Продолжительность каждого отрезка в секундах
            total_segments = math.ceil(file_duration / segment_duration)  # Общее количество отрезков

            # Инициализируем список для хранения текста отрезков
            segment_texts = []

            for segment_index in range(total_segments):
                start = segment_index * segment_duration
                end = min((segment_index + 1) * segment_duration, file_duration)

                with tempfile.NamedTemporaryFile(delete=False) as temp_segment:
                    # Вырезаем отрезок из файла
                    segment = AudioSegment.from_file(input_file)[start * 1000:end * 1000].set_frame_rate(8000)

                    # Сохраняем временный отрезок
                    segment.export(temp_segment.name, format="wav")

                    # Распознаем отрезок и сохраняем текст
                    recognizer = sr.Recognizer()
                    with sr.AudioFile(temp_segment.name) as source:
                        audio_data = recognizer.record(source)
                        segment_text = recognizer.recognize_google(audio_data, language="ru-RU")
                    segment_texts.append(segment_text)

            # Соединяем тексты всех отрезков
            text = ''.join(segment_texts)
        else:
            # Если файл не требует деления, распознаем его как обычно
            recognizer = sr.Recognizer()
            with sr.AudioFile(input_file) as source:
                audio_data = recognizer.record(source)
                text = recognizer.recognize_google(audio_data, language="ru-RU")

        end_time = time.time()
        processing_time = end_time - start_time
        total_time += processing_time

        # Сохраняем распознанный текст в файл
        with open(output_file, 'w') as f:
            f.write(text)
        print(f"Файл {i} обработан и сохранен за {processing_time:.2f} секунд")

    except sr.UnknownValueError:
        print(f"Ошибка при обработке файла {i}: не удалось распознать речь.")
    except sr.RequestError as e:
        print(f"Ошибка при запросе к Google Speech Recognition в файле {i}: {e}")

total_time = total_time / file_number
print(f"Среднее время обработки 1 файла: {total_time:.2f} секунд.")

Файл 1 обработан и сохранен за 7.57 секунд
Файл 2 обработан и сохранен за 3.98 секунд
Файл 3 обработан и сохранен за 5.91 секунд
Файл 4 обработан и сохранен за 3.74 секунд
Файл 5 обработан и сохранен за 1.95 секунд
Файл 6 обработан и сохранен за 2.79 секунд
Файл 7 обработан и сохранен за 1.98 секунд
Файл 8 обработан и сохранен за 2.41 секунд
Файл 9 обработан и сохранен за 5.55 секунд
Файл 10 обработан и сохранен за 2.69 секунд
Файл 11 обработан и сохранен за 8.50 секунд
Файл 12 обработан и сохранен за 8.02 секунд
Файл 13 обработан и сохранен за 5.04 секунд
Файл 14 обработан и сохранен за 4.47 секунд
Файл 15 обработан и сохранен за 2.91 секунд
Файл 16 обработан и сохранен за 8.72 секунд
Файл 17 обработан и сохранен за 3.07 секунд
Файл 18 обработан и сохранен за 5.63 секунд
Файл 19 обработан и сохранен за 4.29 секунд
Файл 20 обработан и сохранен за 1.04 секунд
Файл 21 обработан и сохранен за 4.50 секунд
Файл 22 обработан и сохранен за 2.28 секунд
Файл 23 обработан и сохранен за 3.64 секу

ФУНКЦИЯ ПОЛУЧЕНИЯ ДЛИТЕЛЬНОСТИ И ЧАСТОТЫ АУДИО

In [ ]:
def get_duration(file_path):
    audio = AudioSegment.from_file(file_path)
    duration_in_seconds = len(audio) / 1000  # Получаем длительность в секундах
    return duration_in_seconds


РАСПОЗНОВАНИЕ АУДИО (WHISPER)

ПОДГРУЖАЕМ МОДЕЛЬ WISHPER

In [ ]:
model_whisper = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [01:55<00:00, 26.7MiB/s]


In [ ]:
audio_folder = '/content/drive/MyDrive/Audio/extracted1/Датасет_аудио1'
text_files_path = '/content/drive/MyDrive/Audio/extracted1/whisper1/'
os.makedirs(text_files_path, exist_ok=True)

total_time = 0
file_number = 0

for i in range(1, 502):  # Обрабатываем 502 файла
    input_file = os.path.join(audio_folder, f"{i:03d}.ogg")
    output_file = os.path.join(text_files_path, f"{i:03d}.txt")
    file_number += 1

    # Засекаем время начала обработки файла
    start_time = time.time()

    # Читаем файл
    with open(input_file, 'rb') as f:
        data = f.read()

    # Проверяем размер и длительность файла
    file_size = os.path.getsize(input_file)
    file_duration = get_duration(input_file)

    try:
        # Если файл слишком большой или длинный, делим его на отрезки по 30 секунд
        if file_size > 1024 * 1024 or file_duration > 30:
            segment_duration = 30  # Продолжительность каждого отрезка в секундах
            total_segments = math.ceil(file_duration / segment_duration)  # Общее количество отрезков

            # Инициализируем список для хранения текста отрезков
            segment_texts = []

            for segment_index in range(total_segments):
                start = segment_index * segment_duration
                end = min((segment_index + 1) * segment_duration, file_duration)

                with tempfile.NamedTemporaryFile(delete=False) as temp_segment:
                    # Вырезаем отрезок из файла
                    segment = AudioSegment.from_file(input_file)[start * 1000:end * 1000].set_frame_rate(8000)

                    # Сохраняем временный отрезок
                    segment.export(temp_segment.name, format=".ogg")

                    # Распознаем отрезок и сохраняем текст
                    audio_data = whisper.load_audio(temp_segment.name)
                    result = model_whisper.transcribe(audio_data)
                    segment_text = result["text"]
                    segment_texts.append(segment_text)

            # Соединяем тексты всех отрезков
            text = ''.join(segment_texts)
        else:
            # Если файл не требует деления, распознаем его как обычно
            audio_data = whisper.load_audio(input_file)
            result = model_whisper.transcribe(audio_data)
            text = result["text"]

        end_time = time.time()
        processing_time = end_time - start_time
        total_time += processing_time

        # Сохраняем распознанный текст в файл
        with open(output_file, 'w') as f:
            f.write(text)
        print(f"Файл {i} обработан и сохранен за {processing_time:.2f} секунд")

    except Exception as e:
        print(f"Ошибка при обработке файла {i}: {e}")

total_time = total_time / file_number
print(f"Среднее время обработки 1 файла: {total_time:.2f} секунд.")

Ошибка при обработке файла 1: Encoding failed. ffmpeg/avlib returned error code: 1

Command:['ffmpeg', '-y', '-f', 'wav', '-i', '/tmp/tmpyj3jihhx', '-f', '.ogg', '/tmp/tmpt7df9w_q']

Output from ffmpeg/avlib:

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubbe

СВЕРКА ТАБЛИЧНЫХ ДАННЫХ

СОЗДАЕМ ТАБЛИЦУ С ТЕКСТАМИ

In [ ]:
# Пути к папкам с текстовыми файлами
real_text_folder = '/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Voice_filters/real_text'
speech_recognition_folder = '/content/drive/MyDrive/Audio/extracted1/speech_recognition1'
whisper_folder = '/content/drive/MyDrive/Audio/extracted1/whisper1'

# Создаем списки для хранения текстов из файлов
real_text_list = []
speech_recognition1_text_list = []
whisper1_text_list = []

# Читаем текст из каждого файла и добавляем его в соответствующий список
for i in range(1, 52):
    filename = f"{i:03d}.txt"

    # Чтение из папки с настоящим текстом
    real_file_path = os.path.join(real_text_folder, filename)
    if os.path.exists(real_file_path):
        with open(real_file_path, 'r') as file:
            real_text = file.read()
            real_text_list.append(real_text)
    else:
        real_text_list.append('NAN')


    # Чтение из папки с текстом от Гугл спич рекогнишн
    speech_recognition_path = os.path.join(speech_recognition_folder, filename)
    if os.path.exists(speech_recognition_path):
        with open(speech_recognition_path, 'r') as file:
            speech_recognition_text = file.read()
            speech_recognition1_text_list.append(speech_recognition_text)
    else:
        speech_recognition1_text_list.append('NAN')


    # Чтение из папки с текстом ВИСПЕР
    whisper_path = os.path.join(whisper_folder, filename)
    if os.path.exists(whisper_path):
        with open(whisper_path, 'r') as file:
            whisper_text = file.read()
            whisper1_text_list.append(whisper_text)
    else:
        whisper1_text_list.append('NAN')

In [ ]:
# Создаем датафрейм
df = pd.DataFrame({
    'ИМЯ ФАЙЛА': [f"{i:03d}" for i in range(1, 52)],
    'ИСТИННЫЙ ТЕКСТ': real_text_list,
    'GOOGLE ТЕКСТ': speech_recognition1_text_list,
    'WHISPER ТЕКСТ': whisper1_text_list,
})

СЧИТАЕМ ОШИБКУ Word Error Rate

In [ ]:
# Создаем список для хранения WER значений
wer_speech_recognition1_list = []
wer_whisper1_list = []

# Проходимся по каждой паре текстов и рассчитываем WER
for true_text, recognized_text_speech_recognition, recognized_text_whisper in zip(real_text_list, speech_recognition1_text_list, whisper1_text_list):

    # Рассчитываем WER для Google Speech Recognition
    wer_speech_recognition = round(jiwer.wer(true_text, recognized_text_speech_recognition), 2)
    wer_speech_recognition1_list.append(wer_speech_recognition)

    # Рассчитываем WER для WHISPER
    wer_whisper = round(jiwer.wer(true_text, recognized_text_whisper), 2)
    wer_whisper1_list.append(wer_whisper)

# Добавляем столбцы WER к датафрейму
df['GOOGLE WER'] = wer_speech_recognition1_list
df['WHISPER WER'] = wer_whisper1_list

average_wer_speech_recognition = df['GOOGLE WER'].mean()
average_wer_whisper = df['WHISPER WER'].mean()

In [ ]:
# Выводим первые пять строк датафрейма для проверки
from IPython.display import display

# Установить настройки для вывода всей таблицы
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Вывести таблицу
display(df)

,ИМЯ ФАЙЛА,ИСТИННЫЙ ТЕКСТ,GOOGLE ТЕКСТ,WHISPER ТЕКСТ,GOOGLE WER,WHISPER WER
0,001,"так еду 23 ноября и мне нужно нижнее место не у туалета, где-нибудь посередине вагона.",так еду 23 ноябрянинМне нужно нижнее место не у туалета где-нибудь посередине вагона,NAN,0.33,1.00
1,002,"а я из Уфы Поеду в Кострому, мне нужен билет через неделю но лучше на выходные, так у меня будет на поездку 10.000 руб.",а я из Уфы Поеду в Кострому Мне нужен билет через неделю он лучше на выходные Так у меня будет на поездку 10.000 руб,"Я из Уфы, поеду в Кострому. Мне нужен билет через неделю. Лучше на выходные. Так, у меня будет на поездку 10 тысяч рублей.",0.25,0.58
2,003,"так завтра утром мы едем, 5 человек из Москвы в Петербург, вечером мы выезжаем обратно в Москву и через 3 дня мы едем в Краснодар, также пять человек, найди самые быстрые билеты самое быстрое время.",Так завтра утром мы едем пять человек из Москвы в Петербург вечером мы выезжаем обратно в Москву И через 3 дня мы едем в Краснодар а также пять человек Найди самые быстрые билеты самое быстрое время,"Так, завтра утром мы едем, пять человек из Москвы в Петербург. Вечером мы выезжаем обратно в Москву, и через три дня мы едем в Краснодар. Также пять человек. Найди самый быстрый билет за самое быстрое время.",0.29,0.40
3,004,"так мне нужно добраться из Новосибирска до Москвы, я буду с двумя детьми, у меня багаж две сумки по 10 кг, найди мне билеты на 13 августа.",так Мне нужно добраться из Новосибирска до Москвы Я буду с двумя детьми у меня багаж две сумки по 10 кг Найди мне билеты на 13 августа,так не нужно самолет набраться из новосибирска до москвы я буду с двумя детьми у меня багаж две сумки по 10 килограмм найди мне билеты на 13 августа,0.26,0.30
4,005,"поеду из Архангельска в Тверь 15 октября и 20 октября я из Твери поеду в Омск, подбери билеты.",поеду из Архангельска в Тверь 15 октября и 20 октября я из Твери поеду в Омск подбери билеты,Поеду из Верхангельска в Тверь 15 октября. И 20 октября я из Твери поеду в Омск. Подбери билеты.,0.11,0.33
5,006,"планирую из Новокузнецка выехать 30 декабря в Ростов, из Ростова нужно 8 января вернуться обратно, я буду с мужем и с ребёнком, найди билеты.",планируй из Новокузнецка выехать 30 декабря в Ростов из Ростова нужно 8 января вернуться обратно я буду с мужем И с ребёнком Найди билеты,Планирую из Новокузнецка выехать 30 декабря в Ростов. Из Ростова нужно 8 января вернуться обратно. Я буду с мужем и с ребенком на Эдди-билеты.,0.29,0.33
6,007,"с 1 июля по 1 августа я поеду из Краснодара в Уфу, мне нужны билеты.",с 1 июля по 1 августа я поеду из Краснодара в Уфу Мне нужны билеты,С 1 июля по 1 августа я поеду из Краснодара в Уфу. Мне нужны билеты.,0.20,0.20
7,008,"так найди-ка мне билеты из Сочи в Москву, у меня есть 5.000 и мне нужен билет на сегодня.",так найди-ка мне билеты из Сочи в Москву У меня есть 5.000 А мне нужен билет на сегодня,так на дико не билеты из сочи в москву у меня есть 5000 мне нужен билет на сегодня,0.22,0.44
8,009,"7 января хочу на самолете долететь из Омска в Тобольск, подбери билет, я буду одна, у меня с собой ручная кладь.",7 января 1809 из Омска в Тобольск я буду одна у меня с собой ручная,"7 января хочу на самолете долететь из Умска в Тобольск. Подбери билет, я буду одна, у меня с собой ручная кладь.",0.43,0.14
9,010,"с 14 по 20 февраля я поеду из Твери в Москву и потом обратно, я буду одна, подбери самый дешёвый билет.",с 14 по 20 февраля А я поеду из Твери в Москву И потом обратно я буду одна подбери самый дешёвый билет,С 14 по 20 февраля я поеду из Твери в Москву. И потом обратно я буду одна подбирать самый дешевый билет.,0.24,0.33


СРЕДНИЕ ЗНАЧЕНИЯ

In [ ]:

wer_data = {
    'Метод': ['GOOGLE', 'WHISPER'],
    'Стоимость копеек/штутка': [0, 0],
    'Время распознования, секунд': [3.67, 3.76],
    'Точность распознавания, %': [average_wer_speech_recognition, average_wer_whisper]
}

wer_df = pd.DataFrame(wer_data)
display(wer_df)

,Метод,Стоимость копеек/штутка,"Время распознования, секунд","Точность распознавания, %"
0,GOOGLE,0,3.67,0.344314
1,WHISPER,0,3.76,0.431373


ИЗВЛЕЧЕНИЕ СУЩНОСТЕЙ

УСТАНОВКА НЕОБХОДИМЫХ БИБЛИОТЕК

In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 53.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install pymorphy2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 30.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=9186e257a4fbdbac0be0da002795a1df845496ba284b04cff7212aeadf0ec37e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:

import spacy
import json
import re
import pymorphy2
from datetime import datetime


In [ ]:
morph = pymorphy2.MorphAnalyzer()

ЗАГРУЖАЕМ МОДЕЛЬ ДЛЯ РУССКОГО ЯЗЫКА

In [ ]:
nlp = spacy.load("ru_core_news_sm")

РАБОЧИЕ ПАПКИ

In [ ]:
input_dir = '/content/drive/MyDrive/Audio/extracted1/whisper1'
output_dir = '/content/drive/MyDrive/Audio/extracted/entity'

ОБНУЛЯЕМ ПАПКУ

In [ ]:
import shutil
shutil.rmtree(output_dir)
os.makedirs(output_dir)

ОБРАБАТЫВАЕМ ДАТЫ

In [ ]:
def replace_words_with_numbers(text):
    number_dict = {
        'первое': '1',
        'первого': '1',
        'второе': '2',
        'второго': '2',
        'третье': '3',
        'третьего': '3',
        'четвертое': '4',
        'четвертого': '4',
        'пятое': '5',
        'пятого': '5',
        'шестое': '6',
        'шестого': '6',
        'седьмое': '7',
        'седьмого': '7',
        'восьмое': '8',
        'восьмого': '8',
        'девятое': '9',
        'девятого': '9',
        'десятое': '10',
        'десятого': '10',
        'одиннадцатое': '11',
        'одиннадцатого': '11',
        'двенадцатое': '12',
        'двенадцатого': '12',
        'тринадцатое': '13',
        'тринадцатого': '13',
        'четырнадцатое': '14',
        'четырнадцатого': '14',
        'пятнадцатое': '15',
        'пятнадцатого': '15',
        'шестнадцатое': '16',
        'шестнадцатого': '16',
        'семнадцатое': '17',
        'семнадцатого': '17',
        'восемнадцатое': '18',
        'восемнадцатого': '18',
        'девятнадцатое': '19',
        'девятнадцатого': '19',
        'двадцатое': '20',
        'двадцатого': '20',
        'двадцать первое': '21',
        'двадцать второе': '22',
        'двадцать третье': '23',
        'двадцать четвертое': '24',
        'двадцать пятое': '25',
        'двадцать шестое': '26',
        'двадцать седьмое': '27',
        'двадцать восьмое': '28',
        'двадцать девятое': '29',
        'тридцатое': '30',
        'тридцать первое': '31'
    }

    word_pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, number_dict.keys())) + r')\b', re.IGNORECASE)
    text = word_pattern.sub(lambda x: number_dict[x.group().lower()], text)

    return text

def replace_months_with_numbers(text):
    # Получаем текущий год
    current_year = datetime.now().year

    month_dict = {
        r'\bянвар[ьяе]*\b': f'.01.{current_year}',
        r'\bфеврал[ьяе]*\b': f'.02.{current_year}',
        r'\bмарт[ае]*\b': f'.03.{current_year}',
        r'\bапрел[ьяе]*\b': f'.04.{current_year}',
        r'\bма[йяе]*\b': f'.05.{current_year}',
        r'\bиюн[ьяе]*\b': f'.06.{current_year}',
        r'\bиюл[ьяе]*\b': f'.07.{current_year}',
        r'\bавгуст[ае]*\b': f'.08.{current_year}',
        r'\bсентябр[ьяе]*\b': f'.09.{current_year}',
        r'\bоктябр[ьяе]*\b': f'.10.{current_year}',
        r'\bноябр[ьяе]*\b': f'.11.{current_year}',
        r'\bдекабр[ьяе]*\b': f'.12.{current_year}'
    }


    for month_form, month_number in month_dict.items():
        text = re.sub(r'(\s|^)' + month_form, r'\1' + month_number, text, flags=re.IGNORECASE)

    return text

def process_dates(text):
    # Ищем сочетание цифра пробел точка и две цифры (день месяца)
    date_pattern = re.compile(r'(\b\d)\s*\.\s*(\d{2}\b)')

    # Заменяем совпадения в тексте, удаляя пробел перед точкой
    text = date_pattern.sub(lambda x: '0' + x.group(1) + '.' + x.group(2), text)

    return re.sub(r'(\d)\s*\.\s*(\d{2})', r'\1.\2', text)

def extract_date_from_text(text):
    # Используем регулярное выражение для поиска даты в формате dd.mm.yyyy
    match = re.search(r'\b(\d{2}.\d{2}.\d{4})\b', text)

    # Если найдено совпадение, возвращаем найденную дату
    if match:
        return match.group(1)
    else:
        return datetime.now().strftime('%d.%m.%Y')  # Если дата не найдена, возвращаем текущую дату

def replace_relative_dates(text, base_date):
    # Ищем выражение "через неделю" и заменяем на дату через 7 дней
    text = re.sub(r'\bчерез неделю\b', (datetime.strptime(base_date, '%d.%m.%Y') + timedelta(days=7)).strftime('%d.%m.%Y'), text)

    # Ищем выражение "через неделю" и заменяем на дату через 14 дней
    text = re.sub(r'\bчерез 2 недели\b', (datetime.strptime(base_date, '%d.%m.%Y') + timedelta(days=14)).strftime('%d.%m.%Y'), text)

    # Ищем выражение "через неделю" и заменяем на дату через 21 дней
    text = re.sub(r'\bчерез 3 недели\b', (datetime.strptime(base_date, '%d.%m.%Y') + timedelta(days=21)).strftime('%d.%m.%Y'), text)

    # Ищем выражение "через месяц" и заменяем на дату через 30 дней
    text = re.sub(r'\bчерез месяц\b', (datetime.strptime(base_date, '%d.%m.%Y') + timedelta(days=30)).strftime('%d.%m.%Y'), text)

    # Ищем выражения "через ТУТ ЧИСЛО дней" и заменяем на соответствующие даты
    matches = re.finditer(r'\bчерез (\d+) дн(?:ей|я)\b', text)
    for match in matches:
        days_to_add = int(match.group(1))
        replacement_date = (datetime.strptime(base_date, '%d.%m.%Y') + timedelta(days=days_to_add)).strftime('%d.%m.%Y')
        text = text.replace(match.group(0), replacement_date)

    return text


ФУНКЦИЯ ПЕРЕНОСА ПО ОЧЕРЕДИ СЛОВ С ПРЕДЛОГАМИ "ИЗ" "С" "ОТ" В КОНЕЦ ТЕКСТА

In [ ]:
def rearrange_text_begin(sentence):
    words = sentence.split()
    rearranged_words = []
    preposition_words = []

    i = 0
    while i < len(words):
        if words[i] == 'из' or words[i] == 'с' or words[i] == 'от' or words[i] == 'из-за':
            if i + 1 < len(words):
                preposition_words.extend([words[i], words[i + 1]])
                i += 2
            else:
                preposition_words.extend(words[i:])
                break
        else:
            rearranged_words.append(words[i])
            i += 1

    rearranged_words = preposition_words + rearranged_words
    return ' '.join(rearranged_words)


ФУНКЦИЯ ПЕРЕНОСА ПО ОЧЕРЕДИ СЛОВ С ПРЕДЛОГАМИ "В" "ДО" В КОНЕЦ ТЕКСТА

In [ ]:
def rearrange_text_end(sentence):
    words = sentence.split()
    rearranged_words = []
    preposition_words = []

    i = 0
    while i < len(words):
        if words[i] == 'в' or words[i] == 'до':
            if i + 1 < len(words):
                preposition_words.extend([words[i], words[i + 1]])
                i += 2
            else:
                preposition_words.extend(words[i:])
                break
        else:
            rearranged_words.append(words[i])
            i += 1

    rearranged_words = rearranged_words + preposition_words
    return ' '.join(rearranged_words)


РЕШАЕМ ВОПРОС ГОРОДОВ ЧЕРЕЗ ТИРЕ (ПРОГРАММА ВОСПРИНИМАЕТ ИХ КАК ОДИН ГОРОД)

In [ ]:
def process_hyphens(text):
    prefixes_to_ignore = ["Санкт", "Усть", "Улан", "Южно", "Катав", "Переславль", "Ростов", "Ростов-на", "Комсомольске", "Комсомольск", "Комсомольске-на", "Петропавловск", "Петровск", "Наро", "Горно", "Калача", "Калача-на"]
    words = text.split()

    for i in range(1, len(words)):
        if (
            words[i - 1] not in prefixes_to_ignore
            and '-' in words[i]
            and not any(prefix in words[i] for prefix in prefixes_to_ignore)
        ):
            words[i] = words[i].replace('-', ' ')

    return ' '.join(words)


ФУНКЦИЯ ДЛЯ ИЗВЛЕЧЕНИЯ СУЩНОСТЕЙ

In [ ]:
import os
import json
import re
from datetime import datetime, timedelta
from dateutil import parser

def extract_route_from_directory(directory_path, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)

            try:
                with open(file_path, "r", encoding="utf-8") as file:
                    text = file.read()

                    # Вывод текста файла
                    print(f"Текст:{filename}:{text}")

                    # Поиск и установка дат
                    text = replace_words_with_numbers(text)
                    text = replace_months_with_numbers(text)
                    text = process_dates(text)
                    print(f"Текст:{filename}:{text}")

                    date_result = extract_date_from_text(text)
                    if date_result:
                        print("Найденная дата:", date_result)
                        text = replace_relative_dates(text, date_result)
                    print(f"Текст:{filename}:{text}")

                    # Перестановка слов в предложении
                    text = rearrange_text_begin(text)
                    text = rearrange_text_end(text)
                    text = process_hyphens(text)

                    # Обработка текста с помощью SpaCy
                    doc = nlp(text)

                    # Извлечение и вывод всех локаций
                    all_locations = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == 'LOC']
                    print(f"Найденные локации в {filename}: {all_locations}")

                    # Определение маршрута с учетом контекста перемещения
                    route_info = {}

                    if all_locations:
                        start_location_index = text.find(all_locations[0][0])
                        end_location_index = text.rfind(all_locations[-1][0])

                        route_info["city_from"] = morph.parse(all_locations[0][0])[0].normal_form
                        route_info["city_to"] = morph.parse(all_locations[-1][0])[0].normal_form
                        route_info["city_transit"] = [morph.parse(loc[0])[0].normal_form for loc in all_locations[1:-1]]

                        # Удаление дубликатов
                        route_info["city_transit"] = list(set(route_info["city_transit"]))
                        if route_info["city_to"] == route_info["city_from"]:
                            del route_info["city_to"]
                        if route_info["city_transit"] == [route_info["city_to"]] or route_info["city_transit"] == [route_info["city_from"]]:
                            del route_info["city_transit"]

                        # Добавление условия обратного пути
                        if any(word in text for word in ["обратно", "возвращаюсь", "возвращаемся"]):
                            route_info["city_transit"].append(route_info["city_to"])
                            route_info["city_to"] = route_info["city_from"]

                        # Добавление информации о виде транспорта
                        transport_keywords = ["самолет", "самолеты", "самолёт", "самолёты", "автобус", "автобусы"]
                        # По умолчанию "transport": поезд
                        route_info["transport"] = "поезд"
                        # Проверяем, есть ли в тексте ключевые слова для других видов транспорта
                        for keyword in transport_keywords:
                            if keyword in text.lower():
                                route_info["transport"] = keyword
                                break  # Прерываем цикл, если найдено ключевое слово

                        # Добавление условия наличия части слова "животн", "кошк" или "собак"
                        route_info["animal"] = any(keyword in text.lower() for keyword in ["животн", "кошк", "собак"])

                        # Добавление информации о датах
                        route_info["date_from"] = None
                        route_info["date_transit"] = []

                        # Поиск и добавление дат в формате дд.мм.гггг
                        date_pattern = re.compile(r'\b(\d{1,2}\.\d{1,2}\.\d{4})\b')
                        dates = date_pattern.findall(text)
                        for date in dates:
                            if date not in route_info["date_transit"]:
                                route_info["date_transit"].append(date)
                                if route_info["date_from"] is None:
                                    route_info["date_from"] = date

                        # Если в тексте есть слово "завтра", то устанавливаем завтрашнюю дату в "date_from"
                        if "завтра" in text.lower():
                            tomorrow_date = datetime.now() + timedelta(days=1)
                            route_info["date_from"] = tomorrow_date.strftime('%d.%m.%Y')

                        # Если нет дат в тексте, добавляем текущую дату в "date_from"
                        if route_info["date_from"] is None:
                            route_info["date_from"] = datetime.now().strftime('%d.%m.%Y')

                        # Вывод информации о маршруте
                        print(f"Данные поездки {filename}: {route_info}\n")

                        # Сохранение информации о маршруте в JSON файл
                        output_filename = os.path.join(output_directory, filename.replace(".txt", ".json"))
                        with open(output_filename, "w", encoding="utf-8") as output_file:
                            json.dump(route_info, output_file, ensure_ascii=False, indent=4)

            except Exception as e:
                print(f"Error processing file {filename}: {str(e)}")


In [ ]:
extract_route_from_directory(input_dir, output_dir)

Текст:002.txt: Я из Уфы, поеду в Кострому. Мне нужен билет через неделю. Лучше на выходные. Так, у меня будет на поездку 10 тысяч рублей.
Текст:002.txt: Я из Уфы, поеду в Кострому. Мне нужен билет через неделю. Лучше на выходные. Так, у меня будет на поездку 10 тысяч рублей.
Найденная дата: 07.03.2024
Текст:002.txt: Я из Уфы, поеду в Кострому. Мне нужен билет 14.03.2024. Лучше на выходные. Так, у меня будет на поездку 10 тысяч рублей.
Найденные локации в 002.txt: [('Уфы', 'LOC'), ('Кострому', 'LOC')]
Данные поездки 002.txt: {'city_from': 'уфа', 'city_to': 'кострома', 'city_transit': [], 'transport': 'поезд', 'animal': False, 'date_from': '14.03.2024', 'date_transit': ['14.03.2024']}

Текст:003.txt: Так, завтра утром мы едем, пять человек из Москвы в Петербург. Вечером мы выезжаем обратно в Москву, и через три дня мы едем в Краснодар. Также пять человек. Найди самый быстрый билет за самое быстрое время.
Текст:003.txt: Так, завтра утром мы едем, пять человек из Москвы в Петербург. Вечеро

In [ ]:
import re
from datetime import datetime, timedelta

def extract_date_from_text(text):
    # Используем регулярное выражение для поиска даты в формате dd.mm.yyyy
    match = re.search(r'\b(\d{2}.\d{2}.\d{4})\b', text)

    # Если найдено совпадение, возвращаем найденную дату
    if match:
        return match.group(1)
    else:
        return datetime.now().strftime('%d.%m.%Y')  # Если дата не найдена, возвращаем текущую дату

def replace_relative_dates(text, base_date):
    # Ищем выражение "через неделю" и заменяем на дату через 7 дней
    text = re.sub(r'\bчерез неделю\b', (datetime.strptime(base_date, '%d.%m.%Y') + timedelta(days=7)).strftime('%d.%m.%Y'), text)

    # Ищем выражение "через неделю" и заменяем на дату через 14 дней
    text = re.sub(r'\bчерез 2 недели\b', (datetime.strptime(base_date, '%d.%m.%Y') + timedelta(days=14)).strftime('%d.%m.%Y'), text)

    # Ищем выражение "через неделю" и заменяем на дату через 21 дней
    text = re.sub(r'\bчерез 3 недели\b', (datetime.strptime(base_date, '%d.%m.%Y') + timedelta(days=21)).strftime('%d.%m.%Y'), text)

    # Ищем выражение "через месяц" и заменяем на дату через 30 дней
    text = re.sub(r'\bчерез месяц\b', (datetime.strptime(base_date, '%d.%m.%Y') + timedelta(days=30)).strftime('%d.%m.%Y'), text)

    # Ищем выражения "через ТУТ ЧИСЛО дней" и заменяем на соответствующие даты
    matches = re.finditer(r'\bчерез (\d+) дн(?:ей|я)\b', text)
    for match in matches:
        days_to_add = int(match.group(1))
        replacement_date = (datetime.strptime(base_date, '%d.%m.%Y') + timedelta(days=days_to_add)).strftime('%d.%m.%Y')
        text = text.replace(match.group(0), replacement_date)

    return text

# Пример использования
text_example = "через месяц командировка 05.07.2024 из Москвы в Новосиб Новосибирск оттуда через 2 недели Это примерно до 20.07.2024 летим в Якутию и из Якутии 01.08.2024 найди билет обратно до Москвы"
date_result = extract_date_from_text(text_example)

if date_result:
    print("Найденная дата:", date_result)
    updated_text = replace_relative_dates(text_example, date_result)
    print("Обновленный текст:", updated_text)
else:
    print("Дата не найдена в тексте.")

Найденная дата: 05.07.2024
Обновленный текст: 04.08.2024 командировка 05.07.2024 из Москвы в Новосиб Новосибирск оттуда 19.07.2024 Это примерно до 20.07.2024 летим в Якутию и из Якутии 01.08.2024 найди билет обратно до Москвы


СТАРИЧКИ

In [ ]:
# Список для хранения текстов
texts = []
number = 1
# Путь к папке с файлами
folder_path = "/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Voice_filters/real_text"

# Обрабатываем файлы с 1 по 51
for i in range(number, number+51):
    filename = f"{i:03d}.txt"
    file_path = os.path.join(folder_path, filename)

    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        texts.append(text)

# Вывод для проверки, можно закомментировать после использования
for idx, text in enumerate(texts):
    print(f"{text}")

так еду 23 ноября и мне нужно нижнее место не у туалета, где-нибудь посередине вагона.
а я из Уфы Поеду в Кострому, мне нужен билет через неделю но лучше на выходные, так у меня будет на поездку 10.000 руб.
так завтра утром мы едем, 5 человек из Москвы в Петербург, вечером мы выезжаем обратно в Москву и через 3 дня мы едем в Краснодар, также пять человек, найди самые быстрые билеты самое быстрое время.
так мне нужно добраться из Новосибирска до Москвы, я буду с двумя детьми, у меня багаж две сумки по 10 кг, найди мне билеты на 13 августа.
поеду из Архангельска в Тверь 15 октября и 20 октября я из Твери поеду в Омск, подбери билеты.
планирую из Новокузнецка выехать 30 декабря в Ростов, из Ростова нужно 8 января вернуться обратно, я буду с мужем и с ребёнком, найди билеты.
с 1 июля по 1 августа я поеду из Краснодара в Уфу, мне нужны билеты.
так найди-ка мне билеты из Сочи в Москву, у меня есть 5.000 и мне нужен билет на сегодня.
7 января хочу на самолете долететь из Омска в Тобольск, под

БАЗА ДЛЯ ДООБУЧЕНИЯ NER

In [ ]:
TRAIN_DATA = [
    ("Нужен билет на автобус из Белореченска в Крымск на 3 часа на пятнадцатое число на 3:00 дня на пятнадцатое число ближе к выходу то есть спереди автобус хочу сидеть",
     {"entities": [(20, 33, "city_from"), (49, 55, "city_to"), (63, 71, "date_from"), (87, 93, "time_from")]}),
    ("в выходные Хотелось бы посмотреть билеты на самолёт от Мурманска до Петербурга предложи какие-нибудь вечерние рейсы на трёх пассажиров",
     {"entities": [(22, 30, "city_from"), (34, 41, "city_to")]}),
    ("какие рейсы есть рейсы с Хабаровска на двадцать четвёртое число до Москвы три пассажира прямой без пересадки",
     {"entities": [(22, 31, "city_from"), (35, 40, "date_to"), (44, 49, "city_to")]}),
    ("подбери пожалуйста поезд вечерний с ранним прибытием из Челябинска до Магнитогорска с 25 по 24 сентября",
     {"entities": [(48, 60, "city_from"), (64, 75, "city_to"), (79, 92, "date_from")]}),
    ("а мне с багажом примерно 30 кг нужно из Ивантеевки добраться в Белебей 22 февраля обратно в 31 тогда получается не в тридцатых примерно первого числа 1 марта обратно билет",
     {"entities": [(32, 43, "city_from"), (57, 64, "city_to"), (82, 91, "date_from"), (108, 115, "date_to")]}),
    ("а мне из Кирова в Калужской области нужно доехать до Владимира 04.08 вагон обязательно должен быть не смешанный только для девушек",
     {"entities": [(9, 15, "city_from"), (29, 48, "city_to"), (52, 59, "date_to")]}),
    ("Мне двадцать третьего числа нужно из Феодосии в Чебоксары на 2 дня и потом обратно",
     {"entities": [(11, 24, "city_from"), (28, 37, "city_to"), (41, 54, "date_from")]}),
    ("покажи какие есть варианты улететь третьего числа из Вологды в Иркутск",
     {"entities": [(33, 40, "city_from"), (44, 51, "city_to"), (55, 68, "date_from")]}),
    ("Да надо поехать из Москвы в Питер вдвоём бюджет не больше пяти Я отправка вечером",
     {"entities": [(19, 25, "city_from"), (29, 34, "city_to"), (44, 50, "budget"), (57, 65, "time_of_day")]}),
    ("Сегодня собираюсь ехать с другом из Оренбурга в Самару едем на концерт",
     {"entities": [(31, 39, "city_from"), (43, 48, "city_to"), (52, 59, "activity")]}),
    ("Да надо поехать из Москвы в Казань бюджет не больше четырёх на одного также поездка вечером",
     {"entities": [(19, 25, "city_from"), (29, 34, "city_to"), (44, 51, "budget"), (58, 66, "time_of_day")]}),
    ("раз нужна поездка из Твери в Новосибирск у меня есть 10.000 руб поездка будет числа пятнадцатого где-то",
     {"entities": [(20, 26, "city_from"), (30, 42, "city_to"), (56, 70, "budget"), (81, 96, "date_from")]}),
    ("надо поехать из Москвы в Казань отправка в пятницу вечером в 8:00 бюджет не дороже 5",
     {"entities": [(19, 25, "city_from"), (29, 34, "city_to"), (44, 51, "time_of_day"), (55, 62, "time"), (69, 76, "budget")]}),
    ("Да надо поехать из Москвы в Питер бюджет не больше пяти отправка 8:00",
     {"entities": [(19, 25, "city_from"), (29, 34, "city_to"), (44, 50, "budget"), (57, 64, "time_of_day"), (72, 76, "time")]}),
    ("Короче нам с семьёй надо переехать в Новосибирск из Краснодара да планирую поездку восемнадцатого числа примерно на 15.000 руб на поездку",
     {"entities": [(39, 47, "city_from"), (51, 62, "city_to"), (66, 78, "date_from"), (90, 104, "budget")]}),
    ("через 2 часа надо быть в Новороссийске с Краснодара за 10.000 руб",
     {"entities": [(40, 52, "city_from"), (56, 68, "city_to"), (72, 85, "time"), (95, 109, "budget")]}),
    ("Да надо поехать из Москвы в Питер где-то в районе пятницы наверное бюджет не больше 5.000 отправка в 8 часов",
     {"entities": [(19, 25, "city_from"), (29, 34, "city_to"), (48, 55, "budget"), (65, 73, "time_of_day"), (79, 86, "time")]}),
    ("сгоняли Мы короче с Оренбурга до Уфы на хакатон победили заняли первое место всё чётко ездили втроём кстати",
     {"entities": [(26, 34, "city_from"), (38, 41, "city_to"), (52, 58, "activity"), (74, 87, "outcome")]}),
    ("Короче мне надо из Ростова в блять в Тверь 18 числа 20.000 руб у меня есть Какие есть варианты",
     {"entities": [(20, 27, "city_from"), (31, 34, "city_to"), (38, 48, "date_from"), (52, 65, "budget")]}),
    ("Блядь да тихо ты ну Шахты Темрюк 10.000 руб нужно доехать 18 ноября выезжаю восемнадцатого да",
     {"entities": [(31, 37, "city_from"), (41, 48, "city_to"), (57, 71, "budget"), (87, 97, "date_from")]}),
    ("на самом-то деле доехать из Челябинска в Екатеринбург довольно там быстро",
     {"entities": [(17, 28, "city_from"), (32, 45, "city_to")]}),
    ("У меня друг есть дело из Казани в Нижний Новгород через Чебоксары там очень долго ехать они там втроём ездили на машине заправки долго искали",
     {"entities": [(25, 31, "city_from"), (35, 50, "city_to"), (62, 73, "city_through"), (78, 83, "activity"), (97, 111, "mode_of_transport")]}),
    ("Да разок я ездил из Ижевска в Пермь в целом ничего страшного дороги нормальные",
     {"entities": [(22, 30, "city_from"), (34, 39, "city_to"), (43, 52, "experience"), (58, 67, "road_condition")]}),
    ("В общем хочу заказать билеты из Оренбурга до Москвы летим нас двое а даты примерно может 1 октября 2 октября двадцать третьего года я бы так сказал Вот в это время собираемся",
     {"entities": [(24, 32, "city_from"), (36, 41, "city_to"), (58, 63, "count"), (77, 88, "date_from")]}),
    ("собираемся короче ехать с другом до Москвы с Оренбурга на машине по деньгам это выйдет наверное тысяч на 10 если не 20 хотим ехать примерно через неделю то есть 1 ноября",
     {"entities": [(36, 43, "city_from"), (47, 52, "city_to"), (63, 68, "mode_of_transport"), (84, 101, "budget"), (113, 122, "date_from")]}),
    ("Да надо поехать из Москвы в Питер где-то в районе пятницы наверное бюджет не больше 5.000 отправка в 8 часов",
     {"entities": [(20, 26, "city_from"), (30, 35, "city_to"), (61, 66, "budget"), (82, 88, "time_of_day")]}),
    ("Да надо поехать из Москвы в Питер бюджет не больше пяти отправка 8:00",
     {"entities": [(20, 26, "city_from"), (30, 35, "city_to"), (51, 56, "budget"), (62, 67, "time_of_day")]}),
    ("надо поехать из Москвы в Казань отправка в пятницу вечером в 8:00 бюджет не дороже 5",
     {"entities": [(27, 33, "city_from"), (37, 42, "city_to"), (53, 60, "time_of_day"), (75, 80, "budget")]}),
    ("надо поехать из Москвы в Питер отправка наверное в 8 бюджет не больше 5 да отправка в пятницу вечером и можно засыпать тёплые носочки носочки",
     {"entities": [(20, 26, "city_from"), (30, 35, "city_to"), (51, 56, "time_of_day"), (62, 67, "budget")]}),
    ("Алиса Закажи поезд из Москвы в Питер в 8:00 в пятницу вечером бюджет не дороже пяти результат на почту",
     {"entities": [(27, 33, "city_from"), (37, 42, "city_to"), (56, 63, "time_of_day"), (79, 84, "budget")]}),
     ("так Ну примерно через месяц к Новому году нужно будет ехать из Санкт-петербурга до Кургана прямым рейсом ну и на дорогу наверное понадобится ну тысяч 30 потому что да нужно будет взять и жену с собой и ребёнка",
     {"entities": [(53, 69, "city_from"), (73, 79, "city_to"), (98, 102, "time_of_year"), (142, 148, "budget")]}),
    ("ещё и к тёще же надо съездить а это получается ехать до Белгородской области а ехать нужно через Тулу Курск и различные другие города Ну и тут поездочка обойдётся намного побольше то есть примерно тысяч 50 соответственно набрать расчёты на бензин на остановки на еду ну где-то так",
     {"entities": [(20, 26, "city_from"), (30, 36, "city_to"), (62, 70, "via_city"), (114, 120, "budget")]}),
    ("в принципе надо съездить в Белгород а точно в Белгород надо ехать во к тёще надо а там ехать получается через Москву и Псков Тверь Москва Курск Тула в Тулу поедем и денег надо вот денег там надо умотаться тысяч 50 да на проезд на еду и на бухло а на бухло надо еще отдельно тысяч 30 бухло будет будет",
     {"entities": [(26, 34, "via_city"), (38, 44, "via_city"), (50, 56, "via_city"), (58, 63, "via_city"), (65, 70, "via_city"), (72, 76, "via_city"), (106, 112, "via_city"), (136, 142, "budget")]}),
    ("Короче я планирую поехать но я сейчас в Питере в Санкт-петербурге планирую поехать в Самару но нам надо ещё короче Волгоград и в Казань будет заехать в Казань не знаю наверное сначала в Казань мы поедем ну потом в Волгаград короче со мной поедет Светка Иванченко коллега и наверно сын поедет Ромка не заню вот с собакой пока не разобралась собаку оставлять или с собой брать ну как то так",
     {"entities": [(65, 71, "city_from"), (75, 81, "city_to"), (106, 112, "city_to"), (130, 136, "city_to"), (165, 171, "city_to"), (177, 183, "budget")]}),
    ("я с ребёнком и мужем хочу поехать с Барнаула в Ялту новогодние праздники выезд 28 декабря на неделю найди поездку суммой до 20.000",
     {"entities": [(20, 28, "city_from"), (32, 36, "city_to"), (88, 98, "date_from"), (104, 110, "budget")]}),
    ("нужно добраться из Краснодара в Москву самым быстрым рейсом до 15.000 руб 29 декабря",
     {"entities": [(26, 36, "city_from"), (40, 46, "city_to"), (69, 79, "budget"), (81, 91, "date")]}),
    ("Здравствуйте мне с сыном нужно в Кисловодск на 2 недели неважно за 10.000 руб через 2 дня то есть Пятнадцатого числа",
     {"entities": [(39, 51, "city_to"), (59, 68, "budget"), (74, 84, "date")]}),
    ("бля Короче надо из Семикаракорска за пару дней короче с Семикаракорска семнадцатого числа я выезжаю и мне нужно быть в Краснодаре",
     {"entities": [(29, 41, "city_from"), (55, 65, "date"), (71, 81, "city_to")]}),
    ("через 2 часа надо быть в Новороссийске с Краснодара за 10.000 руб",
     {"entities": [(39, 51, "city_to"), (55, 65, "city_from"), (71, 81, "budget")]}),
    ("Короче мне надо из Ростова в блять в Тверь 18 числа 20.000 руб у меня есть Какие есть варианты",
     {"entities": [(29, 36, "city_from"), (41, 46, "city_to"), (52, 62, "date"), (68, 78, "budget")]}),
    ("Короче нам с семьёй надо переехать в Новосибирск из Краснодара да планирую поездку восемнадцатого числа примерно на 15.000 руб на поездку",
     {"entities": [(40, 50, "city_to"), (54, 64, "city_from"), (74, 84, "date"), (104, 114, "budget")]}),
    ("Блядь да тихо ты ну Шахты Темрюк 10.000 руб нужно доехать 18 ноября выезжаю восемнадцатого да",
     {"entities": [(23, 29, "city_from"), (33, 39, "city_to"), (52, 62, "budget"), (78, 88, "date")]}),
    ("раз нужна поездка из Твери в Новосибирск у меня есть 10.000 руб поездка будет числа пятнадцатого где-то",
     {"entities": [(26, 31, "city_from"), (35, 45, "city_to"), (77, 87, "budget"), (93, 103, "date")]}),
    ("из Батайска я через пару дней то есть пятого числа хочу переехать из Батайска хочу билет на Тверь 10.000 руб",
     {"entities": [(29, 36, "city_from"), (54, 64, "date"), (80, 86, "city_to"), (92, 102, "budget")]}),
    ("из Красноярска в Краснодар 10.000 руб поездка",
     {"entities": [(26, 37, "city_from"), (41, 51, "city_to"), (57, 67, "budget")]}),
    ("Здравствуйте нам с семьёй трое человек нужно переехать из-за Саратова в Магнитогорск 30.000 руб число 15 вот пятнадцатого хотим переехать в Магнитогорск",
     {"entities": [(31, 40, "city_from"), (45, 56, "city_to"), (76, 86, "budget"), (94, 104, "date")]}),
    ("Здравствуйте я с другом еду из Саратова в Тверь Господи блядь в Твери еду а с другом из Саратова бюджет 20.000 руб 21 декабря",
     {"entities": [(31, 40, "city_from"), (45, 50, "city_to"), (66, 76, "budget"), (82, 92, "date")]}),
    ("с 1 по 10 мая хочу поехать в Санкт-петербург заехать в Кронштадт 20 что заехать в Кронштадт с 3 по 5 пятого обратно в Петербург нас будет двое",
     {"entities": [(34, 50, "city_to"), (67, 78, "date_from"), (82, 92, "date_to"), (104, 118, "city_to"), (122, 134, "date_from_return"), (142, 156, "city_from"), (164, 174, "person_count")]}),
    ("а Закажи билеты из Москвы в Курунганьск бюджет до 50 отправка в пятницу вечером",
     {"entities": [(29, 35, "city_from"), (39, 49, "city_to"), (58, 68, "budget"), (74, 88, "time_of_day")]}),
    ("Набери билеты из Казани в Питер с бюджетом в одну сторону до 7.000 руб в пятницу и вечером",
     {"entities": [(22, 28, "city_from"), (32, 37, "city_to"), (53, 62, "budget"), (76, 83, "date"), (87, 95, "time_of_day")]}),
    ("Слушай сегодня поеду я из Рысаево в Орск наверное по пути заеду Новотроицк нужно будет забрать вещи и наверное заправить на полный бак сразу рублей на 1.000 наверное",
     {"entities": [(32, 40, "city_from"), (44, 48, "city_to"), (64, 73, "city_stop"), (99, 110, "budget")]}),
    ("надо короче сгонять из Ясного в Гай посмотреть что там случилось",
     {"entities": [(20, 26, "city_from"), (30, 33, "city_to")]}),
    ("Сегодня собираюсь ехать с другом из Оренбурга в Самару едем на концерт",
     {"entities": [(35, 43, "city_from"), (47, 52, "city_to")]}),
    ("сгоняли Мы короче с Оренбурга до Уфы на хакатон победили заняли первое место всё чётко ездили втроём кстати",
     {"entities": [(22, 30, "city_from"), (34, 37, "city_to")]}),
    ("поехали мы короче с Бузулука до Магнитогорска перед этим ещё остановились в Стерлитамаке поездка выдалась то ещё",
     {"entities": [(23, 31, "city_from"), (35, 45, "city_to"), (64, 76, "city_stop")]}),
    ("на самом-то деле доехать из Челябинска в Екатеринбург довольно там быстро",
     {"entities": [(28, 39, "city_from"), (43, 56, "city_to")]}),
    ("Да разок я ездил из Ижевска в Пермь в целом ничего страшного дороги нормальные",
     {"entities": [(25, 32, "city_from"), (36, 41, "city_to")]}),
    ("У меня друг есть дело из Казани в Нижний Новгород через Чебоксары там очень долго ехать они там втроём ездили на машине заправки долго искали",
     {"entities": [(20, 26, "city_from"), (30, 45, "city_to"), (51, 60, "city_stop")]}),
    ("У меня знакомый кстати из Ярославля сам гонял до Москвы туда-обратно на неделе ничего говорит нормально",
     {"entities": [(32, 43, "city_from"), (47, 53, "city_to")]}),
    ("В общем хочу заказать билеты из Оренбурга до Москвы летим нас двое а даты примерно может 1 октября 2 октября двадцать третьего года я бы так сказал Вот в это время собираемся",
     {"entities": [(37, 45, "city_from"), (49, 54, "city_to"), (88, 104, "date")]}),
    ("собираемся короче ехать с другом до Москвы с Оренбурга на машине по деньгам это выйдет наверное тысяч на 10 если не 20 хотим ехать примерно через неделю то есть 1 ноября",
     {"entities": [(33, 41, "city_to"), (45, 54, "city_from"), (94, 104, "date")]}),
    ("Здорово пацаны планирую отправиться на отдых в Сочи хочу заказать билеты с Оренбурга до Сочи что думаете может быть на 1 июня по деньгам сколько будет тысяч 10 15 нормально нет что скажете планируем втроём полететь",
     {"entities": [(64, 71, "city_from"), (75, 80, "city_to"), (92, 106, "date"), (128, 142, "budget")]}),
    ("Здравствуйте здравствуйте планируем компании полететь в Сочи будьте добры подскажите пожалуйста стоимость авиаперелётов с Оренбурга до Сочи примерно 1 декабря человек нас 5 10 думаем вот отправиться в путешествие новогодние",
     {"entities": [(77, 85, "city_from"), (89, 94, "city_to"), (112, 124, "date"), (132, 136, "person_count")]}),
    ("так короче у нас загрузка под Самарой там загружаемся по 20 т щебня и выезжаем короче этот на получается сейчас так так куда же сейчас блин где же тут написано а вот сначала на Оренбург 10 т потом по 10 т с машины потом на Уфу короче по 10 т и обратно на Самару вот такой рейс у нас будет",
     {"entities": [(22, 29, "city_stop"), (46, 54, "city_from"), (58, 68, "city_to"), (88, 94, "weight"), (99, 105, "cargo_type"), (120, 133, "city_stop")]}),
    ("до Ростова какие есть варианты на эти выходные сгонять вернёмся обратно в понедельник",
     {"entities": [(3, 10, "city_to"), (49, 60, "date")]}),
    ("срочно найди билеты до Красноярска на следующую неделю желательно чтобы сумма небольшая была вот по времени неважно",
     {"entities": [(27, 37, "city_to"), (47, 57, "date")]}),
    ("я собираюсь в командировку с маршрутом Санкт-Петербург Тверь Ярославль Москва в каждом городе у меня запланирована встреча с партнёрами поэтому время строго регламентировано предполагаемая сумма расходов на поездку около 50.000 руб со мной едет ассистент так что необходимо учесть его расходы",
     {"entities": [(34, 51, "city_from"), (55, 60, "city_stop"), (62, 70, "city_stop"), (72, 77, "city_stop"), (79, 85, "city_to"), (137, 149, "budget"), (169, 181, "person_count")]}),
    ("моё имя Алексей Я этнограф организую экспедицию в малоизвестные народные уголки России наша цель село Верхняя Санарка Алтайского края мы планируем изучить местные традиции и обычаи наша команда из пяти человек рассчитывает на бюджет 100.000 руб что должно покрыть транспортные расходы проживание питание на 2 недели",
     {"entities": [(29, 41, "person_name"), (64, 84, "expedition_goal"), (110, 129, "city_to"), (159, 170, "budget"), (194, 206, "person_count")]}),
    ("подбери завтра на утро поезд до Магнитогорска и оттуда нужно улететь в Челябинск через день 5 января обратно самолётом до Адлера Какие есть варианты",
     {"entities": [(28, 40, "city_to"), (64, 77, "city_to"), (93, 102, "date"), (116, 125, "date"), (141, 147, "city_to")]}),
    ("срочно найди мне билет из Питера в Калининград на завтра у ранний рейс мне найди пожалуйста",
     {"entities": [(26, 33, "city_from"), (37, 48, "city_to"), (56, 61, "date"), (69, 80, "time_of_day")]}),
    ("через месяц командировка 5 июля из Москвы в Новосиб Новосибирск оттуда через 2 недели Это примерно до 20 июля летим в Якутию и из Якутии 1 августа найди билет обратно до Москвы",
     {"entities": [(30, 36, "date"), (40, 46, "city_from"), (50, 58, "city_to"), (67, 74, "date"), (96, 109, "city_to"), (115, 123, "date"), (131, 142, "city_from")]}),
    ("Найди пожалуйста на 3 января поезд из Нерюнгри в Хабаровск сутки там переждём и пятого числа найди билет на самолёт Хабаровск-Москва",
     {"entities": [(34, 41, "city_from"), (45, 54, "city_to"), (74, 83, "date"), (105, 116, "city_from"), (120, 127, "city_to")]}),
    ("на 20 октября рейс самолёт Благовещенск-Чита какие есть варианты утром ранний выезд",
     {"entities": [(16, 25, "date"), (31, 45, "city_from"), (49, 53, "city_to"), (59, 67, "time_of_day")]}),
    ("Покажи как добраться с Благовещенска до Красноярска какие есть самолёты на 20 ноября и обратно тот же маршрут но уже 24 ноября",
     {"entities": [(29, 42, "city_from"), (46, 56, "city_to"), (67, 76, "date"), (82, 92, "city_from"), (98, 108, "city_to"), (119, 128, "date")]}),
]

In [ ]:
from spacy.training.example import Example

# Дообучение модели
for text, annotations in TRAIN_DATA:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    nlp.update([example], drop=0.5, losses={})

# Сохранение дообученной модели
nlp.to_disk("/content/drive/MyDrive/NLP_spacy")

In [ ]:
# Загрузка дообученной модели
output_dir = '/content/drive/MyDrive/NLP_spacy'
nlp_my = spacy.load(output_dir)

# Текст для обработки
text = "Какие есть варианты добраться 15 декабря из Краснодара в Новосибирск? Обратный возврат 19 декабря пассажир 1"

# Применение модели к тексту
doc = nlp_my(text)

# Вывод сущностей из текста
for ent in doc.ents:
    print(f"Сущность: {ent.text}, Тип: {ent.label_}")

In [ ]:
nlp = spacy.load("ru_core_news_sm")

# Текст для обработки
text = "Какие есть варианты добраться 15 декабря из Краснодара в Новосибирск? Обратный возврат 19 декабря пассажир 1"

# Применение модели к тексту
doc = nlp(text)

# Вывод сущностей из текста
for ent in doc.ents:
    print(f"Сущность: {ent.text}, Тип: {ent.label_}")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Сущность: Краснодара, Тип: LOC
Сущность: Новосибирск, Тип: LOC


ЧАТ ГПТ АПИ

In [ ]:
!pip install nest_asyncio xmltodict openai==0.28.0 langchain==0.0.281 tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00


In [ ]:
import tiktoken
import getpass
import openai
import tiktoken
from openai.embeddings_utils import distances_from_embeddings
import pandas as pd
import numpy as np
from ast import literal_eval
from langchain.chains.router import MultiRetrievalQAChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import os
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity

from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key


OpenAI API Key:··········


In [ ]:
# инициализируем модель
llm = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo-16k")

In [ ]:
schema = {
    "properties": {
        "city_from": {"type": "string"},
        "date_from": {"type": "string"},
        "time_from": {"type": "string"},
        "city_to": {"type": "string"},
        "transport": {"type": "string"},
        "animal": {"type": "string"},
        "children": {"type": "integer"},
        "baggage": {"type": "string"},
        "comfort": {"type": "string"},
    },
    "required": ["amount"]
}

def extract_entities(content: str, schema: dict):
    # Структурируем текст запроса с пояснениями о необходимых сущностях
    content_with_info = (
        f"Вставляй только данные NONE\n"
        "Если есть, тебе нужно найти сначала город откуда пассажир хочет выехать и вставить в city_from\n"
        "потом потом найти город, куда пассажир хочет ехать: city_to\n"
        "потом найти дату, когда пассажир хочет выехать: date_from\n"
        "потом найти время, когда пассажир хочет выехать: time_from\n"
        "потом посмотреть есть ли животные у пассажира в поездке: animal\n"
        "потом посмотреть есть ли дети у пассажира: children\n"
        "потом посмотреть, есть ли багаж у пассажира: baggage\n"
        "обязательно определить на каком транспорте едет пассажир, автобус, поезд или самолет: transport\n"
        "также нужно определить уровень комфорта СВ, купе, плацкарт, сидячий или вообще без места: comfort"
    )
    return create_extraction_chain(schema=schema, llm=llm).run(content_with_info)

# Пример использования
sample_text = "Мне нужно поехать сегодня из Москвы в Казань"
entities_result = extract_entities(sample_text, schema)
print(entities_result)

[{'city_from': 'NONE', 'date_from': 'NONE', 'time_from': 'NONE', 'city_to': 'NONE', 'transport': 'NONE', 'animal': 'NONE', 'children': 0, 'baggage': 'NONE', 'comfort': 'NONE'}]


In [ ]:
# Схема для извлечения сущностей
schema = {
    "properties": {
        "city_from": {"type": "string"},
        "date_from": {"type": "string"},
        "date_from_t": {"type": "string"},
        "time_from": {"type": "string"},
        "city_to": {"type": "string"},
        "date_to": {"type": "string"},
        "date_to_t": {"type": "string"},
        "time_to": {"type": "string"},
        "amount": {"type": "integer"},
        "transit": {"type": "integer"},
        "transport": {"type": "string"},
        "company": {"type": "string"},
        "animal": {"type": "string"},
        "children": {"type": "integer"},
        "baggage": {"type": "string"},
        "level": {"type": "string"},
        "comfort": {"type": "string"},
        "insurance": {"type": "string"}
    },
    "required": ["amount"]
}

def extract_entities(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

# Пример использования
sample_text = "Посмотри 3 билета, я поеду на поезде с 2 детьми из Москвы в Казань 4 августа часов на 8 вечера с собакой и кошкой и еще будет огромный рюкзак хочу поехать компанией РЖД исключительно в плацкарте"
entities_result = extract_entities(sample_text, schema)
print(entities_result)


[{'city_from': 'Москва', 'date_from': '4 августа', 'date_from_t': 'часов', 'time_from': '8 вечера', 'city_to': 'Казань', 'date_to': '', 'date_to_t': '', 'time_to': '', 'amount': 3, 'transit': 0, 'transport': 'поезде', 'company': 'РЖД', 'animal': 'собакой и кошкой', 'children': 2, 'baggage': 'огромный рюкзак', 'level': '', 'comfort': 'плацкарте', 'insurance': ''}]
